# Hypothesis-7 Hybrid Gradient Toy Model

1次元ポテンシャル $G(v)=\frac{1}{v + i\varepsilon}$ に対し、
通常勾配とHypothesis-7型ハイブリッド勾配の収束速度を比較する。

In [ ]:
import math
import torch
import matplotlib.pyplot as plt

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def run_optimizer(alpha: float, steps: int = 400, lr: float = 0.05):
    # vを学習し、目標G*に合わせる
    v = torch.tensor(2.0, device=device)
    eps = torch.tensor(0.1, device=device)
    target = torch.tensor(0.5 + 0.0j, device=device)
    losses = []
    for _ in range(steps):
        G = 1.0 / (v + 1j * eps)
        loss = torch.abs(G - target) ** 2
        losses.append(float(loss))
        # 理論勾配: dG/dv = -1/(v + i eps)^2
        grad_theory = -1.0 / (v + 1j * eps) ** 2
        grad_loss = 2 * (G - target) * grad_theory
        # Hypothesis-7勾配: 逆二乗を安定化して混合
        denom = (v + 1j * eps) ** 2
        denom_mag = torch.abs(denom)
        denom = torch.where(denom_mag < 1e-3, denom / (denom_mag + 1e-9) * 1e-3, denom)
        grad_h7 = 2 * (G - target) * (1.0 / (denom + 1e-6))
        grad = (1 - alpha) * grad_loss + alpha * grad_h7
        v = v - lr * grad.real  # 実数パラメータのみ更新
    return losses

loss_plain = run_optimizer(alpha=0.0)
loss_hybrid = run_optimizer(alpha=0.5)
iters = list(range(len(loss_plain)))

In [ ]:
plt.figure(figsize=(6,4))
plt.semilogy(iters, loss_plain, label="GD (alpha=0.0)")
plt.semilogy(iters, loss_hybrid, label="Hybrid (alpha=0.5)")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Hypothesis-7 Hybrid vs Plain Gradient")
plt.legend()
plt.grid(True, which="both", ls=":")
plt.tight_layout()
plt.show()

In [ ]:
def first_below(losses, threshold=1e-3):
    for i, l in enumerate(losses):
        if l < threshold:
            return i
    return len(losses)

plain_iter = first_below(loss_plain)
hybrid_iter = first_below(loss_hybrid)
print(f"Iterations to loss<1e-3 -> plain: {plain_iter}, hybrid: {hybrid_iter}")